## Kernel: Python 3.8

In [1]:
from dotenv import dotenv_values
from datetime import timedelta, datetime

import pandas as pd
import requests
import locale

## Get Authority

In [4]:
config = dotenv_values(".env") # get values from .env file
locale.setlocale(locale.LC_ALL, "id") # set local language

def login() -> str:
    try:
        url = 'https://api.kampusmerdeka.kemdikbud.go.id/user/auth/login/mbkm'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
        }
        account = {
            'email': config.get('EMAIL'),
            'password': config.get('PASSWORD')
        }
        response = requests.post(url, json=account, headers=headers)
        token = response.json()['data'].get('access_token')
    
    except Exception:
        raise ValueError(response.json()["error"]["message"])
    
    return token, account

(token, account) = login()
headers = dict()
headers["Accept"] = 'application/json'
headers["Authorization"] = f'Bearer {token}'
headers["User-Agent"] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'

print(f'login to {account["email"]} success')

login to hazlanqodri2020work@gmail.com success


## 1. Fetch ALL Mitra

In [5]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=20&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
reports = response.json().get('data')

print(reports)

[{'id': '6a7bbbab-7508-11ed-b2a2-1a198d2f4c14', 'name': '2D/3D Animator Intern', 'activity_name': 'BNI Digital Talent Thematic Internship', 'total': 13, 'start_duration': '2023-02-16T07:00:00+07:00', 'end_duration': '2023-06-30T07:00:00+07:00', 'start_registration': '2022-12-29T07:00:00+07:00', 'end_registration': '2023-01-20T07:00:00+07:00', 'credits_count': 20, 'activity_type': 'OFFLINE', 'location': 'Kota Jakarta Barat', 'location_kotakab_code': '016200', 'mitra_id': '1434cd57-6295-441a-898a-7294119a96d7', 'certified': True, 'logo': 'https://storage.googleapis.com/kampusmerdeka_kemdikbud_go_id/mitra/mitra_642f3ae3-5162-4f40-aba6-e0c819df5f56.jpg', 'mitra_name': 'BNI'}, {'id': '7659b95a-6429-11ed-b5cc-0a223dcf0f66', 'name': 'Academic Officers Intern', 'activity_name': 'Zenius Professionals Practice Program Angkatan 4', 'total': 6, 'start_duration': '2023-02-13T07:00:00+07:00', 'end_duration': '2023-06-30T07:00:00+07:00', 'start_registration': '2022-12-22T07:00:00+07:00', 'end_registr